In [1]:
###'STAGE 1: AMENITY ANALYSIS' & 'STAGE 2: LISTING CLUSTERING'###

In [2]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import chardet
import gzip
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# =========================
# OUTPUT CONFIG
# =========================
output_dir = r"C:\Users\USER\Downloads\ICC in Digital Platforms\STAGE 1&2_outputs"
os.makedirs(output_dir, exist_ok=True)

def save_figure(fig, filename, dpi=300):
    """Save matplotlib figure to output_dir."""
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath, dpi=dpi, bbox_inches='tight')
    print(f"[Saved figure] {filepath}")

def save_dataframe(df, filename, index=True):
    """Save DataFrame as CSV to output_dir."""
    filepath = os.path.join(output_dir, filename)
    df.to_csv(filepath, index=index, encoding='utf-8-sig')
    print(f"[Saved table] {filepath}")

def append_log(message, filename="icc_log.txt"):
    """Append a line of text to a log file in output_dir."""
    filepath = os.path.join(output_dir, filename)
    with open(filepath, "a", encoding="utf-8") as f:
        f.write(str(message) + "\n")


# File paths (using your existing paths)
files = {
    "reviews": r"C:\Users\USER\Downloads\ICC in Digital Platforms\reviews.csv",
    "neighbourhoods_csv": r"C:\Users\USER\Downloads\ICC in Digital Platforms\neighbourhoods.csv",
    "listings": r"C:\Users\USER\Downloads\ICC in Digital Platforms\listings.csv",
    "neighbourhoods_geojson": r"C:\Users\USER\Downloads\ICC in Digital Platforms\neighbourhoods.geojson",
    "calendar": r"C:\Users\USER\Downloads\ICC in Digital Platforms\calendar.csv.gz"
}

# Your existing helper functions
def detect_encoding(file_path, n_bytes=10000):
    """Detect the file encoding by reading the first n_bytes."""
    with open(file_path, 'rb') as f:
        raw_data = f.read(n_bytes)
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    if encoding is None or encoding.lower() == 'ascii':
        encoding = 'latin1'
    return encoding

def safe_read_csv(file_path, is_gzip=False, **kwargs):
    """Safely read CSV files with encoding detection."""
    encoding = detect_encoding(file_path)
    print(f"Detected encoding for {file_path}: {encoding}")
    
    try:
        if is_gzip:
            with gzip.open(file_path, 'rt', encoding=encoding, errors='replace') as f:
                return pd.read_csv(f, on_bad_lines='skip', **kwargs)
        else:
            with open(file_path, 'r', encoding=encoding, errors='replace') as f:
                return pd.read_csv(f, on_bad_lines='skip', **kwargs)
    except Exception as e:
        print(f"Error reading {file_path} with encoding {encoding}: {e}")
        if is_gzip:
            with gzip.open(file_path, 'rt', encoding='latin1', errors='replace') as f:
                return pd.read_csv(f, on_bad_lines='skip', **kwargs)
        else:
            with open(file_path, 'r', encoding='latin1', errors='replace') as f:
                return pd.read_csv(f, on_bad_lines='skip', **kwargs)


class ICCAnalysisPipeline:
    """Complete pipeline for Information-Context Complementarity analysis"""
    
    def __init__(self):
        self.df_listings = None
        self.df_reviews = None
        self.gdf_neighbourhoods = None
        self.amenity_categories = self._define_amenity_categories()
        
    def _define_amenity_categories(self):
        """Define the 11-category amenity taxonomy based on research"""
        return {
            'Accessibility': ['step-free access', 'accessible', 'wheelchair', 'elevator', 'wide doorway'],
            'Bathroom_Personal': ['shampoo', 'conditioner', 'body soap', 'hot water', 'hair dryer', 
                                'towels', 'toilet paper'],
            'Bedding_Comfort': ['bed linens', 'extra pillows', 'blankets', 'essentials', 'hangers'],
            'Business_Work': ['wifi', 'laptop workspace', 'ethernet', 'dedicated workspace'],
            'Climate_Control': ['air conditioning', 'heating', 'ceiling fan', 'portable fans'],
            'Essential_Tech': ['tv', 'cable tv', 'sound system', 'speaker'],
            'Kitchen_Dining': ['kitchen', 'refrigerator', 'microwave', 'oven', 'stove', 'dishwasher',
                             'dishes', 'cooking basics', 'dining table', 'coffee maker', 'wine glasses'],
            'Outdoor_Recreation': ['balcony', 'patio', 'garden', 'pool', 'hot tub', 'bbq grill',
                                 'outdoor furniture', 'beach essentials'],
            'Safety_Security': ['smoke alarm', 'carbon monoxide alarm', 'fire extinguisher',
                              'first aid kit', 'security cameras', 'lock'],
            'Transportation': ['free parking', 'paid parking', 'ev charger', 'bike'],
            'Family_Pets': ['crib', 'high chair', 'pack n play', 'baby bath', 'pets allowed',
                          'cat litter box', 'dog bowls']
        }
    
    def load_data(self, files_dict):
        """Load all datasets"""
        print("Loading datasets...")
        
        # Load main datasets
        self.df_reviews = safe_read_csv(files_dict["reviews"])
        self.df_listings = safe_read_csv(files_dict["listings"])
        self.gdf_neighbourhoods = gpd.read_file(files_dict["neighbourhoods_geojson"])
        
        print(f"Loaded {len(self.df_listings)} listings")
        print(f"Loaded {len(self.df_reviews)} reviews")
        print(f"Loaded {len(self.gdf_neighbourhoods)} neighbourhoods")
        
    def preprocess_data(self):
        """Clean and preprocess the data"""
        print("Preprocessing data...")
        
        # Convert price to numeric
        if 'price' in self.df_listings.columns:
            self.df_listings['price'] = pd.to_numeric(self.df_listings['price'], errors='coerce')
            self.df_listings['log_price'] = np.log(self.df_listings['price'])
        
        # Create log transformations for key variables
        if 'accommodates' in self.df_listings.columns:
            self.df_listings['log_accommodates'] = np.log(self.df_listings['accommodates'] + 1)
        if 'bedrooms' in self.df_listings.columns:
            self.df_listings['log_bedrooms'] = np.log(self.df_listings['bedrooms'] + 1)
        
        # Clean amenities field
        if 'amenities' in self.df_listings.columns:
            self.df_listings['amenities'] = self.df_listings['amenities'].fillna('')
        
        # Remove listings with missing critical information
        critical_cols = ['price', 'longitude', 'latitude']
        available_cols = [col for col in critical_cols if col in self.df_listings.columns]
        self.df_listings = self.df_listings.dropna(subset=available_cols)
        
        print(f"After cleaning: {len(self.df_listings)} listings")
        
    def stage1_amenity_analysis(self):
        """Stage 1: SNA-Based Feature Engineering and Amenity Community Detection"""
        print("\n=== Stage 1: Amenity Analysis ===")
        
        # Parse amenities
        self.df_listings = self._parse_amenities(self.df_listings)
        
        # Create amenity co-occurrence network
        amenity_network = self._create_amenity_network()
        
        # Detect communities
        communities = self._detect_amenity_communities(amenity_network)
        
        # Visualize network
        self._visualize_amenity_network(amenity_network, communities)
        
        # Save derived data
        amenity_cols = [c for c in self.df_listings.columns if c.startswith("Profile_Category_")]
        save_dataframe(self.df_listings[amenity_cols + ['price', 'log_price', 'longitude', 'latitude']],
                       "stage1_amenity_features.csv")
        
        return amenity_network, communities
    
    def _parse_amenities(self, df):
        """Parse and categorize amenities"""
        print("Parsing amenities...")
        
        # Initialize category count columns
        for category in self.amenity_categories:
            df[f'Profile_Category_{category}_Count'] = 0
        
        # Parse amenities string and count by category
        for idx, row in df.iterrows():
            if pd.isna(row['amenities']):
                continue
                
            amenities_list = str(row['amenities']).lower().split(',')
            amenities_list = [a.strip().strip('"{}[]') for a in amenities_list]
            
            for category, keywords in self.amenity_categories.items():
                count = 0
                for keyword in keywords:
                    count += sum(1 for amenity in amenities_list if keyword in amenity)
                df.at[idx, f'Profile_Category_{category}_Count'] = count
        
        return df
    
    def _create_amenity_network(self):
        """Create amenity co-occurrence network"""
        print("Creating amenity co-occurrence network...")
        
        # Extract unique amenities
        all_amenities = set()
        for amenities_str in self.df_listings['amenities'].dropna():
            amenities_list = str(amenities_str).lower().split(',')
            amenities_list = [a.strip().strip('"{}[]') for a in amenities_list]
            all_amenities.update(amenities_list)
        
        # Filter amenities (remove very rare and very common)
        amenity_counts = {}
        for amenity in all_amenities:
            count = self.df_listings['amenities'].str.contains(amenity, case=False, na=False).sum()
            amenity_counts[amenity] = count
        
        total_listings = len(self.df_listings)
        filtered_amenities = [
            amenity for amenity, count in amenity_counts.items()
            if 0.01 * total_listings <= count <= 0.95 * total_listings
        ]
        
        print(f"Filtered to {len(filtered_amenities)} amenities for network analysis")
        
        # Save amenity frequency table
        amenity_freq_df = (pd.Series(amenity_counts, name="count")
                           .to_frame()
                           .assign(freq=lambda d: d['count'] / total_listings)
                           .sort_values("count", ascending=False))
        save_dataframe(amenity_freq_df, "amenity_frequencies.csv")
        
        # Create co-occurrence matrix
        G = nx.Graph()
        G.add_nodes_from(filtered_amenities)
        
        # Add edges based on co-occurrence
        for _, row in self.df_listings.iterrows():
            if pd.isna(row['amenities']):
                continue
            amenities_list = str(row['amenities']).lower().split(',')
            amenities_list = [a.strip().strip('"{}[]') for a in amenities_list]
            present_amenities = [a for a in amenities_list if a in filtered_amenities]
            
            # Add edges between all pairs of amenities in this listing
            for i in range(len(present_amenities)):
                for j in range(i + 1, len(present_amenities)):
                    if G.has_edge(present_amenities[i], present_amenities[j]):
                        G[present_amenities[i]][present_amenities[j]]['weight'] += 1
                    else:
                        G.add_edge(present_amenities[i], present_amenities[j], weight=1)
        
        # Save basic network stats
        net_stats = {
            "n_nodes": G.number_of_nodes(),
            "n_edges": G.number_of_edges(),
            "density": nx.density(G)
        }
        append_log(f"Amenity network stats: {net_stats}")
        
        return G
    
    def _detect_amenity_communities(self, G):
        """Detect communities using Louvain algorithm"""
        print("Detecting amenity communities...")
        
        try:
            import networkx.algorithms.community as nx_comm
            communities = nx_comm.louvain_communities(G, seed=42)
        except Exception as e:
            append_log(f"Louvain failed, fallback to single community: {e}")
            communities = [set(G.nodes())]
        
        print(f"Detected {len(communities)} communities")
        for i, community in enumerate(communities):
            print(f"Community {i}: {len(community)} amenities")
            if len(community) <= 10:  # Show small communities
                print(f"  Sample amenities: {list(community)[:5]}")
        
        # Save community assignments
        comm_records = []
        for i, comm in enumerate(communities):
            for amen in comm:
                comm_records.append({"amenity": amen, "community_id": i})
        comm_df = pd.DataFrame(comm_records)
        save_dataframe(comm_df, "amenity_communities.csv")
        
        return communities
    
    def _visualize_amenity_network(self, G, communities):
        """Visualize the amenity network"""
        fig = plt.figure(figsize=(15, 10))
        ax = fig.add_subplot(111)
        
        # Create layout
        pos = nx.spring_layout(G, k=1, iterations=50)
        
        # Color nodes by community
        colors = plt.cm.Set3(np.linspace(0, 1, len(communities)))
        node_colors = []
        for node in G.nodes():
            for i, community in enumerate(communities):
                if node in community:
                    node_colors.append(colors[i])
                    break
            else:
                node_colors.append('gray')
        
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=50, alpha=0.7, ax=ax)
        nx.draw_networkx_edges(G, pos, alpha=0.3, width=0.5, ax=ax)
        
        ax.set_title("Amenity Co-occurrence Network with Community Detection", fontsize=16)
        ax.axis('off')
        fig.tight_layout()
        save_figure(fig, "fig_amenity_network.png")
        plt.close(fig)
    
    def stage2_clustering_analysis(self):
        """Stage 2: Multi-Stage Clustering of Listings"""
        print("\n=== Stage 2: Listing Clustering ===")
        
        # Prepare features for clustering
        feature_columns = [col for col in self.df_listings.columns 
                          if col.startswith('Profile_Category_') and col.endswith('_Count')]
        
        X = self.df_listings[feature_columns].fillna(0)
        
        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Find optimal clustering
        best_solution, results_df = self._find_optimal_clustering(X_scaled, feature_columns)
        save_dataframe(results_df, "clustering_model_comparison.csv")
        
        # Bootstrap stability analysis
        stability_results = self._bootstrap_stability_analysis(X_scaled, best_solution)
        
        # Assign final clusters
        self.df_listings['Final_Cluster'] = best_solution['labels']
        
        # Profile clusters
        cluster_profiles = self._profile_clusters(feature_columns)
        cluster_profile_df = pd.DataFrame(cluster_profiles).T
        save_dataframe(cluster_profile_df, "cluster_profiles.csv")
        
        # Visualize clustering results
        self._visualize_clustering_results(X_scaled, best_solution['labels'], feature_columns)
        
        return cluster_profiles, stability_results
    
    def _find_optimal_clustering(self, X_scaled, feature_columns):
        """Find optimal clustering solution"""
        print("Finding optimal clustering solution...")
        
        results = []
        
        for algorithm in ['kmeans', 'agglomerative']:
            for k in range(2, 8):
                if algorithm == 'kmeans':
                    clusterer = KMeans(n_clusters=k, random_state=42, n_init=10)
                else:
                    clusterer = AgglomerativeClustering(n_clusters=k)
                
                labels = clusterer.fit_predict(X_scaled)
                
                silhouette = silhouette_score(X_scaled, labels)
                ch_score = calinski_harabasz_score(X_scaled, labels)
                db_score = davies_bouldin_score(X_scaled, labels)
                
                results.append({
                    'algorithm': algorithm,
                    'k': k,
                    'silhouette': silhouette,
                    'ch_score': ch_score,
                    'db_score': db_score,
                    'labels': labels
                })
        
        results_df = pd.DataFrame(results)
        
        # Normalize scores and calculate composite score
        for col in ['silhouette', 'ch_score', 'db_score']:
            if results_df[col].max() != results_df[col].min():
                if col == 'db_score':
                    results_df[col + '_norm'] = 1 - (results_df[col] - results_df[col].min()) / (results_df[col].max() - results_df[col].min())
                else:
                    results_df[col + '_norm'] = (results_df[col] - results_df[col].min()) / (results_df[col].max() - results_df[col].min())
            else:
                results_df[col + '_norm'] = 1.0
        
        results_df['composite_score'] = (results_df['silhouette_norm'] + results_df['ch_score_norm'] + results_df['db_score_norm']) / 3
        
        # Find best solution
        best_idx = results_df['composite_score'].idxmax()
        best_solution = results_df.loc[best_idx].to_dict()
        
        print(f"Best solution: {best_solution['algorithm']} with k={int(best_solution['k'])}")
        print(f"Composite score: {best_solution['composite_score']:.3f}")
        
        append_log(f"Best clustering solution: {best_solution}")
        return best_solution, results_df
    
    def _bootstrap_stability_analysis(self, X_scaled, best_solution, n_bootstrap=100):
        """Bootstrap stability analysis"""
        print("Performing bootstrap stability analysis...")
        
        k = int(best_solution['k'])
        original_labels = best_solution['labels']
        ari_scores = []
        
        for i in range(n_bootstrap):
            bootstrap_indices = np.random.choice(len(X_scaled), len(X_scaled), replace=True)
            X_bootstrap = X_scaled[bootstrap_indices]
            
            if best_solution['algorithm'] == 'kmeans':
                clusterer = KMeans(n_clusters=k, random_state=i, n_init=10)
            else:
                clusterer = AgglomerativeClustering(n_clusters=k)
            
            bootstrap_labels = clusterer.fit_predict(X_bootstrap)
            original_bootstrap_labels = original_labels[bootstrap_indices]
            
            ari = adjusted_rand_score(original_bootstrap_labels, bootstrap_labels)
            ari_scores.append(ari)
        
        mean_ari = np.mean(ari_scores)
        std_ari = np.std(ari_scores)
        
        print(f"Bootstrap stability - Mean ARI: {mean_ari:.3f} ± {std_ari:.3f}")
        append_log(f"Bootstrap ARI mean={mean_ari:.3f}, std={std_ari:.3f}")
        
        # Save ARI distribution
        ari_df = pd.DataFrame({"bootstrap_iter": range(n_bootstrap), "ARI": ari_scores})
        save_dataframe(ari_df, "clustering_bootstrap_ari.csv")
        
        return {'ari_scores': ari_scores, 'mean_ari': mean_ari, 'std_ari': std_ari}
    
    def _profile_clusters(self, feature_columns):
        """Profile the clusters"""
        print("Profiling clusters...")
        
        cluster_profiles = {}
        
        for cluster_id in sorted(self.df_listings['Final_Cluster'].unique()):
            cluster_data = self.df_listings[self.df_listings['Final_Cluster'] == cluster_id]
            
            profile = {
                'size': len(cluster_data),
                'share': len(cluster_data) / len(self.df_listings),
                'avg_amenities': cluster_data[feature_columns].sum(axis=1).mean(),
                'feature_means': cluster_data[feature_columns].mean().to_dict()
            }
            
            cluster_profiles[cluster_id] = profile
            
            print(f"\nCluster {cluster_id}:")
            print(f"  Size: {profile['size']} ({profile['share']:.1%})")
            print(f"  Average amenities: {profile['avg_amenities']:.1f}")
        
        return cluster_profiles
    
    def _visualize_clustering_results(self, X_scaled, labels, feature_columns):
        """Visualize clustering results"""
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X_scaled)
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # PCA scatter plot
        scatter = axes[0, 0].scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis', alpha=0.6)
        axes[0, 0].set_title('PCA Projection of Listings by Cluster')
        axes[0, 0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
        axes[0, 0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
        plt.colorbar(scatter, ax=axes[0, 0])
        
        # Cluster size distribution
        cluster_counts = pd.Series(labels).value_counts().sort_index()
        axes[0, 1].bar(range(len(cluster_counts)), cluster_counts.values, color='skyblue')
        axes[0, 1].set_title('Cluster Size Distribution')
        axes[0, 1].set_xlabel('Cluster')
        axes[0, 1].set_ylabel('Number of Listings')
        
        # Feature heatmap
        cluster_means = pd.DataFrame()
        for cluster_id in sorted(np.unique(labels)):
            cluster_mask = labels == cluster_id
            means = pd.Series(X_scaled[cluster_mask].mean(axis=0), 
                              index=[col.replace('Profile_Category_', '').replace('_Count', '') 
                                     for col in feature_columns])
            cluster_means[f'Cluster_{cluster_id}'] = means
    
        sns.heatmap(cluster_means.T, annot=True, cmap='RdBu_r', center=0, 
                    cbar_kws={'label': 'Standardized Feature Value'}, ax=axes[1, 0])
        axes[1, 0].set_title('Standardized Feature Values by Cluster')
        axes[1, 0].set_xlabel('')
        axes[1, 0].set_ylabel('Clusters')
        
        axes[1, 1].axis('off')
        
        plt.tight_layout()
        save_figure(fig, "fig_clustering_results.png")
        plt.close(fig)

    def generate_comprehensive_report(self):
        """Generate a comprehensive analysis report and save text version"""
        print("\n=== COMPREHENSIVE ICC ANALYSIS REPORT ===")
        print("="*50)
        
        lines = []
        lines.append("=== COMPREHENSIVE ICC ANALYSIS REPORT ===")
        lines.append("="*50)
        
        lines.append(f"Dataset Overview:")
        lines.append(f"- Total listings analyzed: {len(self.df_listings)}")
        lines.append(f"- Price range: ${self.df_listings['price'].min():.0f} - ${self.df_listings['price'].max():.0f}")
        lines.append(f"- Average price: ${self.df_listings['price'].mean():.0f}")
        
        if 'Final_Cluster' in self.df_listings.columns:
            cluster_summary = self.df_listings['Final_Cluster'].value_counts().sort_index()
            lines.append("\nListing Clusters:")
            for cluster, count in cluster_summary.items():
                pct = count / len(self.df_listings) * 100
                lines.append(f"- Cluster {cluster}: {count} listings ({pct:.1f}%)")
        
        amenity_cols = [col for col in self.df_listings.columns 
                       if col.startswith('Profile_Category_') and col.endswith('_Count')]
        lines.append("\nAmenity Categories Summary:")
        for col in amenity_cols:
            avg_count = self.df_listings[col].mean()
            category_name = col.replace('Profile_Category_', '').replace('_Count', '')
            lines.append(f"- {category_name}: {avg_count:.2f} average amenities")
        
        lines.append("\nAnalysis completed successfully!")
        lines.append("="*50)
        
        report_text = "\n".join(lines)
        print(report_text)
        
        # Save report text
        report_path = os.path.join(output_dir, "icc_comprehensive_report.txt")
        with open(report_path, "w", encoding="utf-8") as f:
            f.write(report_text)
        print(f"[Saved report] {report_path}")


# Main execution function
def run_stage1_stage2_analysis():
    """Run only Stage 1 and Stage 2 analysis"""
    
    pipeline = ICCAnalysisPipeline()
    
    try:
        pipeline.load_data(files)
        pipeline.preprocess_data()
        
        amenity_network, communities = pipeline.stage1_amenity_analysis()
        cluster_profiles, stability_results = pipeline.stage2_clustering_analysis()
        pipeline.generate_comprehensive_report()
        
        print("\nStage 1 & 2 Analysis completed successfully!")
        
        return pipeline
        
    except Exception as e:
        print(f"Analysis failed with error: {e}")
        append_log(f"Pipeline failed: {e}")
        import traceback
        traceback.print_exc()
        return None


if __name__ == "__main__":
    results = run_stage1_stage2_analysis()
    
    if results:
        print("\nAnalysis completed. Results stored in 'results' variable.")
        print("Access the processed data via: results.df_listings")
    else:
        print("Analysis failed. Please check the error messages above and icc_log.txt.")

Loading datasets...
Detected encoding for C:\Users\USER\Downloads\ICC in Digital Platforms\reviews.csv: latin1
Detected encoding for C:\Users\USER\Downloads\ICC in Digital Platforms\listings.csv: latin1
Loaded 2928 listings
Loaded 71411 reviews
Loaded 18 neighbourhoods
Preprocessing data...
After cleaning: 2927 listings

=== Stage 1: Amenity Analysis ===
Parsing amenities...
Creating amenity co-occurrence network...
Filtered to 168 amenities for network analysis
[Saved table] C:\Users\USER\Downloads\ICC in Digital Platforms\STAGE 1&2_outputs\amenity_frequencies.csv
Detecting amenity communities...
Detected 2 communities
Community 0: 55 amenities
Community 1: 113 amenities
[Saved table] C:\Users\USER\Downloads\ICC in Digital Platforms\STAGE 1&2_outputs\amenity_communities.csv
[Saved figure] C:\Users\USER\Downloads\ICC in Digital Platforms\STAGE 1&2_outputs\fig_amenity_network.png
[Saved table] C:\Users\USER\Downloads\ICC in Digital Platforms\STAGE 1&2_outputs\stage1_amenity_features.csv

In [3]:
###'STEP 3: ESDA/MGWR' & 'STEP 4: ML VALIDATION (XGBoost & SHAP)'###

In [4]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from math import ceil
import warnings

# Spatial Analysis Libraries
try:
    from esda.getisord import G_Local
    from esda.moran import Moran
    from libpysal.weights import KNN
    from mgwr.gwr import GWR, MGWR
    from mgwr.sel_bw import Sel_BW
except ImportError:
    print("CRITICAL: Please install spatial libraries: pip install esda libpysal mgwr")

# Machine Learning Libraries
from scipy.stats import gaussian_kde
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
import shap

# Configuration
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'DejaVu Sans'  # Standard font for English
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 8)

# ==============================================================================
# CONFIGURATION & PATHS
# ==============================================================================
CONFIG = {
    "base_path": r"C:\Users\USER\Downloads\ICC in Digital Platforms",
    "input_file": "listings_Add_Amenity.xlsx",
    "geojson_file": "neighbourhoods.geojson",
    "output_dir_name": "STAGE 3&4_outputs",
    # Hong Kong Bounding Box
    "bounds": {'min_lon': 113.83, 'max_lon': 114.41, 'min_lat': 22.15, 'max_lat': 22.57}
}

class HongKongSpatialAnalysis:
    def __init__(self, config):
        self.config = config
        self.base_path = config['base_path']
        self.output_dir = os.path.join(self.base_path, config['output_dir_name'])
        
        # Create output directory
        os.makedirs(self.output_dir, exist_ok=True)
        
        self.df = None
        self.gdf = None
        self.gdf_boundaries = None
        self.category_vars = []
        
        print("=== Hong Kong Airbnb Spatial Analysis Pipeline Initialized ===")
        print(f"Output Directory: {self.output_dir}")

    # ==========================================================================
    # 1. DATA LOADING & PREPROCESSING
    # ==========================================================================
    def load_and_preprocess(self):
        print("\n[Phase 1] Loading and Preprocessing Data...")
        
        file_path = os.path.join(self.base_path, self.config['input_file'])
        geojson_path = os.path.join(self.base_path, self.config['geojson_file'])
        
        # 1. Load Excel
        try:
            self.df = pd.read_excel(file_path)
            print(f"  - Loaded Raw Data: {self.df.shape}")
        except FileNotFoundError:
            raise FileNotFoundError(f"Input file not found: {file_path}")

        # 2. Load GeoJSON
        try:
            self.gdf_boundaries = gpd.read_file(geojson_path)
            print(f"  - Loaded Boundaries: {len(self.gdf_boundaries)} districts")
        except Exception as e:
            print(f"  - Warning: Could not load GeoJSON ({e}). Maps will lack boundaries.")

        # 3. Cleaning
        # Drop rows without coordinates or target variables
        subset_cols = ['latitude', 'longitude', 'log_price', 'review_scores_rating']
        self.df.dropna(subset=subset_cols, inplace=True)
        
        # Identify Amenity Category Variables
        self.category_vars = [col for col in self.df.columns 
                              if col.startswith('Profile_Category_') and col.endswith('_Count') 
                              and col != 'Profile_Category_Other_Count']
        
        # Impute missing numeric values with median (for modeling stability)
        numeric_cols = self.df.select_dtypes(include=np.number).columns
        for col in numeric_cols:
            if self.df[col].isnull().any():
                median_val = self.df[col].median()
                self.df[col].fillna(median_val, inplace=True)

        # 4. Create GeoDataFrame
        self.gdf = gpd.GeoDataFrame(
            self.df, 
            geometry=gpd.points_from_xy(self.df.longitude, self.df.latitude),
            crs='EPSG:4326'
        )

        # 5. Spatial Filtering (Hong Kong Bounds)
        b = self.config['bounds']
        self.gdf = self.gdf.cx[b['min_lon']:b['max_lon'], b['min_lat']:b['max_lat']].copy()
        
        print(f"  - Final Cleaned Spatial Data: {self.gdf.shape}")
        print(f"  - Amenity Categories Found: {len(self.category_vars)}")

    # ==========================================================================
    # 2. STEP 3-1: ESDA (KDE & Hotspot)
    # ==========================================================================
    def run_esda_analysis(self):
        print("\n[Phase 2] Step 3-1: Exploratory Spatial Data Analysis (ESDA)...")
        
        # 타겟 변수 설정
        target_vars = ['log_price', 'review_scores_rating'] + self.category_vars
        
        # 1) KDE 분석
        self._perform_kde(target_vars)
        
        # 2) Hotspot Analysis (Getis-Ord Gi*)
        hotspot_results = self._perform_hotspot(target_vars)
        
        # 3) 시각화
        self._visualize_hotspots(hotspot_results)
        self._create_interactive_maps(hotspot_results)

    def _perform_kde(self, vars_to_analyze):
        print("  - Running KDE Analysis...")
        b = self.config['bounds']
        xx, yy = np.mgrid[b['min_lon']:b['max_lon']:100j, b['min_lat']:b['max_lat']:100j]
        positions = np.vstack([xx.ravel(), yy.ravel()])
        
        kde_results = {}
        
        for var in vars_to_analyze:
            if var not in self.gdf.columns: continue
            
            try:
                values = self.gdf[var].values
                coords = np.vstack([self.gdf.longitude, self.gdf.latitude])
                # Weighted KDE
                kernel = gaussian_kde(coords, weights=values)
                density = np.reshape(kernel(positions).T, xx.shape)
                kde_results[var] = density
            except Exception:
                continue
        
        # Visualize KDE
        if kde_results:
            n = len(kde_results)
            cols = 3
            rows = ceil(n / cols)
            fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
            axes = axes.flatten()
            
            for i, (var, density) in enumerate(kde_results.items()):
                ax = axes[i]
                title = var.replace("Profile_Category_", "").replace("_Count", "")
                
                # Plot density
                im = ax.contourf(xx, yy, density, levels=15, cmap='hot', alpha=0.85)
                
                # Overlay boundaries
                if self.gdf_boundaries is not None:
                    self.gdf_boundaries.plot(ax=ax, edgecolor='white', facecolor='none', linewidth=0.5)
                
                ax.set_title(title, fontsize=10)
                ax.axis('off')
                
            # Hide unused subplots
            for j in range(i + 1, len(axes)):
                axes[j].axis('off')
                
            plt.suptitle("Kernel Density Estimation (KDE) Heatmaps", fontsize=16, y=1.02)
            plt.tight_layout()
            plt.savefig(os.path.join(self.output_dir, "ESDA_KDE_Heatmaps.png"), dpi=300, bbox_inches='tight')
            plt.close()

    def _perform_hotspot(self, vars_to_analyze):
        print("  - Running Hotspot Analysis (Getis-Ord Gi*)...")
        results = {}
        
        from libpysal.weights import KNN
        from esda import G_Local
        
        # Spatial Weights (KNN)
        w = KNN.from_dataframe(self.gdf, k=8)
        w.transform = 'r'
        
        for var in vars_to_analyze:
            if var not in self.gdf.columns:
                continue
            
            # float64 변환
            y = self.gdf[var].astype(np.float64).values
            
            # n_jobs=1로 Numba 오류 회피
            gi = G_Local(y, w, transform='B', permutations=999, n_jobs=1)
            
            results[var] = {
                'z_score': gi.z_sim,
                'p_value': gi.p_sim,
                'hot': (gi.p_sim < 0.05) & (gi.z_sim > 1.96),
                'cold': (gi.p_sim < 0.05) & (gi.z_sim < -1.96)
            }
        return results

    def _visualize_hotspots(self, results):
        if not results: return
        
        n = len(results)
        cols = 3
        rows = ceil(n / cols)
        fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
        axes = axes.flatten()
        
        for i, (var, res) in enumerate(results.items()):
            ax = axes[i]
            title = var.replace("Profile_Category_", "").replace("_Count", "")
            
            # Background
            if self.gdf_boundaries is not None:
                self.gdf_boundaries.plot(ax=ax, color='#f0f0f0', edgecolor='white')
            
            # Non-significant points
            ax.scatter(self.gdf.longitude, self.gdf.latitude, c='lightgray', s=1, alpha=0.3)
            
            # Hot spots (Red)
            hot_mask = res['hot']
            if hot_mask.any():
                ax.scatter(self.gdf.loc[hot_mask, 'longitude'], self.gdf.loc[hot_mask, 'latitude'], 
                           c='red', s=5, label='Hot Spot')
                
            # Cold spots (Blue)
            cold_mask = res['cold']
            if cold_mask.any():
                ax.scatter(self.gdf.loc[cold_mask, 'longitude'], self.gdf.loc[cold_mask, 'latitude'], 
                           c='blue', s=5, label='Cold Spot')
            
            ax.set_title(title, fontsize=10)
            ax.axis('off')

        for j in range(i + 1, len(axes)):
            axes[j].axis('off')
            
        plt.suptitle("Hotspot Analysis (Getis-Ord Gi*)", fontsize=16, y=1.02)
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, "ESDA_Hotspots.png"), dpi=300, bbox_inches='tight')
        plt.close()

    def _create_interactive_maps(self, results):
        print("  - Generating Interactive Maps...")
        center = [22.3193, 114.1694]
        
        for var, res in results.items():
            m = folium.Map(location=center, zoom_start=11, tiles='CartoDB positron')
            
            # Add GeoJSON boundary if available
            if self.gdf_boundaries is not None:
                folium.GeoJson(
                    self.gdf_boundaries,
                    style_function=lambda x: {'color': 'black', 'weight': 0.5, 'fillOpacity': 0}
                ).add_to(m)
            
            # Add Hot/Cold spots
            hot_group = folium.FeatureGroup(name='Hot Spots')
            cold_group = folium.FeatureGroup(name='Cold Spots')
            
            # Filter data for mapping
            hot_df = self.gdf[res['hot']]
            cold_df = self.gdf[res['cold']]
            
            for _, row in hot_df.iterrows():
                folium.CircleMarker(
                    [row.latitude, row.longitude], radius=3, color='red', fill=True, fill_opacity=0.7,
                    popup=f"{var}: Hot Spot"
                ).add_to(hot_group)
                
            for _, row in cold_df.iterrows():
                folium.CircleMarker(
                    [row.latitude, row.longitude], radius=3, color='blue', fill=True, fill_opacity=0.7,
                    popup=f"{var}: Cold Spot"
                ).add_to(cold_group)
                
            hot_group.add_to(m)
            cold_group.add_to(m)
            folium.LayerControl().add_to(m)
            
            clean_name = var.replace("Profile_Category_", "").replace("_Count", "")
            m.save(os.path.join(self.output_dir, f"Interactive_Map_{clean_name}.html"))

    # ==========================================================================
    # 3. STEP 3-2: SPATIAL MODELING (MGWR)
    # ==========================================================================
    def run_spatial_modeling(self):
        print("\n[Phase 3] Step 3-2: Spatial Modeling (MGWR)...")
        
        # Define models
        models = {
            'Price_Model': {
                'dep': 'log_price',
                'indep': [
                    'log_accommodates', 'log_bedrooms', 'host_is_superhost',
                    'Profile_Category_Business_Work_Count',
                    'Profile_Category_Transportation_Count',
                    'Profile_Category_Safety_Security_Count',
                    'Profile_Category_Accessibility_Count',
                    'Profile_Category_Essential_Tech_Count',
                    'Profile_Category_Kitchen_Dining_Count',
                    'Profile_Category_Bathroom_Personal_Count',
                    'Profile_Category_Outdoor_Recreation_Count',
                    'Profile_Category_Bedding_Comfort_Count',
                    'Profile_Category_Climate_Control_Count'
                ]
            },
            'Rating_Model': {
                'dep': 'review_scores_rating',
                'indep': [
                    'host_is_superhost', 'host_response_rate',
                    'review_scores_cleanliness', 'review_scores_communication',
                    'review_scores_checkin', 'review_scores_location', 'review_scores_value'
                ]
            }
        }
        
        for model_name, config in models.items():
            self._execute_mgwr_pipeline(model_name, config['dep'], config['indep'])

    def _execute_mgwr_pipeline(self, model_name, dep_var, indep_vars):
        print(f"\n  --- Processing {model_name} ({dep_var}) ---")
        
        # Prepare Data
        y = self.gdf[dep_var].values.reshape(-1, 1)
        X = self.gdf[indep_vars].values
        coords = list(zip(self.gdf.longitude, self.gdf.latitude))
        
        # Standardize
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        X_scaled = scaler_X.fit_transform(X)
        y_scaled = scaler_y.fit_transform(y)
        
        # 3.1 OLS & Moran's I
        print("    > Running OLS and checking residuals...")
        # Simple OLS for residual check
        X_ols = np.hstack([np.ones(y_scaled.shape), X_scaled])
        betas = np.linalg.inv(X_ols.T @ X_ols) @ X_ols.T @ y_scaled
        residuals = y_scaled - X_ols @ betas
        
        w = KNN.from_dataframe(self.gdf, k=8)
        w.transform = 'r'
        moran = Moran(residuals, w)
        print(f"      OLS Residual Moran's I: {moran.I:.4f} (p-value: {moran.p_sim:.4f})")
        
        # 3.2 MGWR
        print("    > Running MGWR (This may take time)...")
        selector = Sel_BW(coords, y_scaled, X_scaled, multi=True, spherical=True)
        bw = selector.search()
        print(f"      Selected Bandwidths: {bw}")
        
        mgwr_model = MGWR(coords, y_scaled, X_scaled, selector=selector, spherical=True)
        results = mgwr_model.fit()
        print(f"      MGWR R2: {results.R2:.4f}")
        
        # 3.3 Visualize Coefficients
        self._visualize_mgwr_coeffs(model_name, results, indep_vars)

    def _visualize_mgwr_coeffs(self, model_name, results, var_names):
        print(f"    > Visualizing Coefficients for {model_name}...")
        
        n = len(var_names)
        cols = 3
        rows = ceil(n / cols)
        fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
        axes = axes.flatten()
        
        for i, var in enumerate(var_names):
            ax = axes[i]
            # Extract coefficients (column i in params)
            coeffs = results.params[:, i]
            
            # Plot
            if self.gdf_boundaries is not None:
                self.gdf_boundaries.plot(ax=ax, color='whitesmoke', edgecolor='black', linewidth=0.3)
            
            sc = ax.scatter(self.gdf.longitude, self.gdf.latitude, c=coeffs, cmap='RdBu_r', s=5, alpha=0.8)
            plt.colorbar(sc, ax=ax, fraction=0.046, pad=0.04)
            
            clean_title = var.replace("Profile_Category_", "").replace("_Count", "")
            ax.set_title(f"Coeff: {clean_title}", fontsize=9)
            ax.axis('off')
            
        for j in range(i + 1, len(axes)):
            axes[j].axis('off')
            
        plt.suptitle(f"MGWR Local Coefficients: {model_name}", fontsize=16, y=1.02)
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, f"MGWR_Coeffs_{model_name}.png"), dpi=300, bbox_inches='tight')
        plt.close()

    # ==========================================================================
    # 4. STEP 4: ML VALIDATION (XGBoost & SHAP)
    # ==========================================================================
    def run_ml_validation(self):
        print("\n[Phase 4] Step 4: ML Validation (XGBoost & SHAP)...")
        
        # We will focus on the Price Model for validation example
        target = 'log_price'
        features = [
            'log_accommodates', 'log_bedrooms', 'host_is_superhost',
            'Profile_Category_Business_Work_Count',
            'Profile_Category_Transportation_Count',
            'Profile_Category_Safety_Security_Count',
            'Profile_Category_Accessibility_Count',
            'Profile_Category_Essential_Tech_Count',
            'Profile_Category_Kitchen_Dining_Count',
            'Profile_Category_Bathroom_Personal_Count',
            'Profile_Category_Outdoor_Recreation_Count',
            'Profile_Category_Bedding_Comfort_Count',
            'Profile_Category_Climate_Control_Count'
        ]
        
        print(f"  - Training XGBoost for {target}...")
        
        X = self.gdf[features]
        y = self.gdf[target]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # XGBoost Regressor (Updated parameters for newer versions)
        model = xgb.XGBRegressor(
            objective='reg:squarederror',
            n_estimators=1000,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            early_stopping_rounds=50,
            random_state=42,
            n_jobs=-1
        )
        
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
        print(f"    > Test R2 Score: {model.score(X_test, y_test):.4f}")
        
        # SHAP Analysis
        print("  - Calculating SHAP Values...")
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X)
        
        # 1. Summary Plot (Beeswarm)
        plt.figure()
        shap.summary_plot(shap_values, X, show=False)
        plt.title(f"SHAP Summary: {target}", fontsize=14)
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, f"SHAP_Summary_{target}.png"), dpi=300, bbox_inches='tight')
        plt.close()
        
        # 2. Importance Bar Plot
        plt.figure()
        shap.summary_plot(shap_values, X, plot_type="bar", show=False)
        plt.title(f"SHAP Feature Importance: {target}", fontsize=14)
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_dir, f"SHAP_Importance_{target}.png"), dpi=300, bbox_inches='tight')
        plt.close()

    # ==========================================================================
    # MAIN EXECUTION
    # ==========================================================================
    def run_pipeline(self):
        try:
            self.load_and_preprocess()
            self.run_esda_analysis()
            self.run_spatial_modeling()
            self.run_ml_validation()
            print("\n=== Analysis Pipeline Completed Successfully ===")
            print(f"Results saved in: {self.output_dir}")
        except Exception as e:
            print(f"\nCRITICAL ERROR: {e}")
            import traceback
            traceback.print_exc()

if __name__ == "__main__":
    # Initialize and run
    analysis = HongKongSpatialAnalysis(CONFIG)
    analysis.run_pipeline()

=== Hong Kong Airbnb Spatial Analysis Pipeline Initialized ===
Output Directory: C:\Users\USER\Downloads\ICC in Digital Platforms\STAGE 3&4_outputs

[Phase 1] Loading and Preprocessing Data...
  - Loaded Raw Data: (2928, 136)
  - Loaded Boundaries: 18 districts
  - Final Cleaned Spatial Data: (2927, 137)
  - Amenity Categories Found: 10

[Phase 2] Step 3-1: Exploratory Spatial Data Analysis (ESDA)...
  - Running KDE Analysis...
  - Running Hotspot Analysis (Getis-Ord Gi*)...
  - Generating Interactive Maps...

[Phase 3] Step 3-2: Spatial Modeling (MGWR)...

  --- Processing Price_Model (log_price) ---
    > Running OLS and checking residuals...
      OLS Residual Moran's I: 0.2187 (p-value: 0.0010)
    > Running MGWR (This may take time)...


Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

      Selected Bandwidths: [  49.  241.  280. 1463. 1706.  157. 2925.  259. 1775. 2676.  241.  785.
 2926. 2504.]


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

      MGWR R2: 0.7022
    > Visualizing Coefficients for Price_Model...

  --- Processing Rating_Model (review_scores_rating) ---
    > Running OLS and checking residuals...
      OLS Residual Moran's I: 0.0077 (p-value: 0.1620)
    > Running MGWR (This may take time)...


Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

      Selected Bandwidths: [ 199. 2083.   85.   58.   48.   53.   58.   44.]


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

      MGWR R2: 0.9359
    > Visualizing Coefficients for Rating_Model...

[Phase 4] Step 4: ML Validation (XGBoost & SHAP)...
  - Training XGBoost for log_price...
    > Test R2 Score: 0.5579
  - Calculating SHAP Values...

=== Analysis Pipeline Completed Successfully ===
Results saved in: C:\Users\USER\Downloads\ICC in Digital Platforms\STAGE 3&4_outputs


In [5]:
#